In [1]:
import os
import numpy as np
import pandas as pd
from scipy.misc.pilutil import imresize
from glob import glob
from utils import get_itk_array

In [2]:
nii_path = glob(r'./data_0614/*.nii.gz')

In [3]:
nii_path

['./data_0614\\2020035031_0130_3051_20200130091644_2.nii.gz',
 './data_0614\\2020035031_0130_3051_20200130091644_4.nii.gz',
 './data_0614\\2020035031_0202_3051_20200202100727_2.nii.gz',
 './data_0614\\2020035031_0202_3051_20200202100727_4.nii.gz',
 './data_0614\\2020035031_0205_3051_20200205095731_2.nii.gz',
 './data_0614\\2020035031_0205_3051_20200205095731_4.nii.gz',
 './data_0614\\2020035031_0208_3051_20200208093928_2.nii.gz',
 './data_0614\\2020035031_0208_3051_20200208093928_4.nii.gz',
 './data_0614\\2020035031_0211_3051_20200211112831_2.nii.gz',
 './data_0614\\2020035031_0211_3051_20200211112831_4.nii.gz',
 './data_0614\\2020035031_0214_Lung_Screening_20200214133449_202.nii.gz',
 './data_0614\\2020035031_0214_Lung_Screening_20200214133449_203.nii.gz',
 './data_0614\\2020035031_0224_Chest_Thorax_20200224102911_2.nii.gz',
 './data_0614\\2020035031_0224_Chest_Thorax_20200224102911_4.nii.gz',
 './data_0614\\2020035031_0310_Chest_Thorax_20200310094633_2.nii.gz',
 './data_0614\\2020035

In [104]:
test_path = nii_path[254:254+10]

In [105]:
test_path

['./data_0614\\2020036291_0226_3050_20200226235401_2.nii.gz',
 './data_0614\\2020036291_0226_3050_20200226235401_4.nii.gz',
 './data_0614\\2020036291_0229_3051_20200229104921_2.nii.gz',
 './data_0614\\2020036291_0229_3051_20200229104921_4.nii.gz',
 './data_0614\\2020036291_0304_3051_20200304145339_2.nii.gz',
 './data_0614\\2020036291_0304_3051_20200304145339_4.nii.gz',
 './data_0614\\2020036291_0312_Chest_Thorax_20200312104101_2.nii.gz',
 './data_0614\\2020036291_0312_Chest_Thorax_20200312104101_4.nii.gz',
 './data_0614\\2020036291_0326_Chest_Thorax_20200326095432_2.nii.gz',
 './data_0614\\2020036291_0326_Chest_Thorax_20200326095432_4.nii.gz']

In [106]:
tag = 1
meta_data = []
for name in test_path:
    nii, spacing = get_itk_array(name)
    patientID = os.path.basename(name).split('.')[0].split('_')[0]
    print('matrix shape:\t', nii.shape)
    nii[nii < -1000] = -1000
    nii[nii > 512] = 512
    nii = nii - np.min(nii)
    nii = nii * 1.0 / np.max(nii)
    if tag == 1:
        total = nii
        tag = 0
    else:
        total = np.concatenate((total, nii), axis=0)
    meta_data.append({'patientID': patientID, 'filename': name, 'slice': nii.shape[0], 'spacing': spacing, 'shape': nii.shape})

matrix shape:	 (77, 512, 512)
matrix shape:	 (474, 512, 512)
matrix shape:	 (77, 512, 512)
matrix shape:	 (560, 512, 512)
matrix shape:	 (77, 512, 512)
matrix shape:	 (560, 512, 512)
matrix shape:	 (71, 512, 512)
matrix shape:	 (438, 512, 512)
matrix shape:	 (73, 512, 512)
matrix shape:	 (451, 512, 512)


In [107]:
slices = total.shape[0]
total_all = np.zeros((slices, 256, 256))  # 初始化
for i in range(slices):
    total_all[i, :, :] = imresize(total[i, :, :], (256, 256))  # 缩小尺寸
if np.max(total_all) > 1:
    print('sacle of nii is changed after imresize!\n Now, normlise:')
    # normalise to 0-1
    total_all = total_all - np.min(total_all)
    total_all = total_all * 1.0 / np.max(total_all)

C:\Users\hasee007\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  after removing the cwd from sys.path.


sacle of nii is changed after imresize!
 Now, normlise:


In [108]:
npy_name = name.split('\\')[-1].split('.')[0].split('_')[0]
np.save('{}.npy'.format(npy_name), total_all)
df = pd.DataFrame(meta_data)
df.to_csv('{}.csv'.format(npy_name))
print('matrix shape:\t', total_all.shape)  # (301, 512, 512)

matrix shape:	 (2858, 256, 256)
